# SYP project

Sentiment analysis.

## Phase 1: Baseline for Sentiment classification
Classify weather music reviews are positive or negative.

### Imports

In [1]:
import pandas as pd
import re

### Read in files

In [2]:
!pwd

/mnt/c/Users/sabrina/Dropbox/learning/bsc_data_science/4_2022_spring/2nd-year-project/2yp_sentiment_analysis/code


In [3]:
# https://www.delftstack.com/howto/linux/unzip-gz-file-linux/

# !gzip -d -k ../data/raw/music_reviews_dev.json.gz
# !gzip -d -k ../data/raw/music_reviews_train.json.gz
# !gzip -d -k ../data/raw/music_reviews_test_masked.json.gz 

In [4]:
# fixing trailing data error https://www.statology.org/valueerror-trailing-data/

data_train = pd.read_json('../data/raw/music_reviews_train.json', lines=True)
data_dev = pd.read_json('../data/raw/music_reviews_dev.json', lines=True)
data_test_masked = pd.read_json('../data/raw/music_reviews_test_masked.json', lines=True)

### EDA
#### Training data

In [5]:
data_train.isnull().sum()

verified              0
reviewTime            0
reviewerID            0
asin                  0
style             14113
reviewText           54
summary              31
unixReviewTime        0
sentiment             0
id                    0
vote              83795
image             99644
dtype: int64

In [6]:
data_train['sentiment'].value_counts()

positive    60783
negative    39217
Name: sentiment, dtype: int64

In [14]:
data_test_masked.head(3)

,verified,reviewTime,reviewerID,asin,reviewText,summary,unixReviewTime,sentiment,id,style,vote,image
0,True,"10 24, 2017",A2HAJB8L9NVYTZ,B007Y1AMHE,ok,ok,1508803200,_,0,NaN,NaN,NaN
1,True,"04 8, 2015",AD78RH9JWBDEU,B007Y1AMHE,Its awesome,love it,1428451200,_,1,NaN,NaN,NaN
2,True,"03 3, 2015",A5UNQFT0JQ8B,B007Y1AMHE,"great, really good!",Five Stars,1425340800,_,2,NaN,NaN,NaN


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = data_train['reviewText'].fillna('na')

In [9]:
vectorizer = TfidfVectorizer()
corpus_vec = vectorizer.fit_transform(corpus)

In [10]:
vectorizer.get_feature_names_out()


# print(corpus_vec.shape)

array(['00', '000', '0000', ..., 'zzzzzzzzzzzzzzzz',
       'zzzzzzzzzzzzzzzzzzzzzz',
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz'],
      dtype=object)

# Functions

In [11]:
def tokenizer(sentence):
    """
    Function to find all tokens in a given sentence.
    """
    tok = re.compile('[\'\"]|[A-Za-z]+|[.?!:\'\"]+')
    
    return tok.findall(sentence)

In [12]:
data_train['reviewTokens'] = data_train['reviewText'].fillna('<NA>').apply(tokenizer)

In [13]:
data_train['reviewTokens']

0        [So, creative, !, Love, his, music, the, words...
1        [This, tape, can, hardly, be, understood, and,...
2        [Buy, the, CD, ., Do, not, buy, the, MP, album...
3        [I, love, Dallas, Holms, music, and, voice, !,...
4        [Great, memories, of, my, early, years, in, Ch...
                               ...                        
99995                   [Great, old, song, ., Love, it, .]
99996    [Daniel, Smith, has, no, more, command, of, th...
99997       [I, ', m, enjoying, her, album, very, much, !]
99998    [These, digital, files, are, corrupted, ., The...
99999    [I, saw, the, Tempest, live, performed, by, th...
Name: reviewTokens, Length: 100000, dtype: object

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class tokenizer2(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        if X['Direction'].dtypes != 'float64':
            X['reviewTokens'] = X['reviewText'].fillna('<NA>').apply(tokenizer)
        return X['reviewTokens']

# create the pipeline
pipeline = Pipeline([
    ('tokenize', tokenizer2()),
    ('scaler', StandardScaler()), 
    ('SVM', svm.SVR(C=25, epsilon=5))
    # ('log_reg', LinearRegression())
    ])